# DEMANDA INTERMITENTE (Version 1)

**Micro-Pasos Propuestos**
1. Registry de métricas: definir MAE, sMAPE, MASE, RMSSE y constructor de tablas "overall" y "por paso" en formato largo.
2. Splitter único: generar folds para estrategias *expanding*, *rolling* y *mix*.
3. Runner de folds: conectar el splitter a 4 modelos base (SF_NAIVE, SF_SEASONALNAIVE, SF_CROSTON, PID_Croston), ejecutar varios folds y consolidar métricas (media sobre folds).
4. Tablas anchas: reutilizar build_overall_table y build_step_tables para informes por horizonte/modelo.
5. Persistencia: guardar métricas y predicciones en Parquet por run_id.
6. Gráfica de inspección: train tail + test + forecasts por modelo (opcional).
7. Orquestador ligero: bucle sobre {df_*, H} y {estrategia} con logging y metadatos.
8. Añaidr más modelos: PID_SBA/SBJ/TSB, AutoARIMA/AutoETS/SES/HES/LES/ADIDA (cuando toque).
9. Validación 3-way y búsqueda (cuando entremos a ML/DL): bloque de tuning y metadatos ampliados.

---

**¿Qué es un "fold" exactamente?**
En backtesting temporal, un fold es una partición concreta **(train, test)** respetando el orden temporal.
Ejemplos:
- **Expanding**: el *train* crece (desde un mínimo) y el *test* es siempre los siguientes **H** pasos.
- **Rolling**: el *train* tiene ventana fija deslizante y el *test* sigue siendo los siguientes **H** pasos. Se generan **varios folds** avanzando la frontera temporal; las métricas finales suelen agregarse (p.ej. **media sobre folds**) para estimar rendimiento fuera de muestra de forma más robusta.
## 1. *REGISTRY* DE MÉTRICAS (Modular y Extensible)

- Firma estándar: metric(y_true, y_pred, **kwargs) -> float
- Incluye: MAE, sMAPE (estable con ceros), MASE y RMSSE (muy útiles para comparar entre series).
- Constructor de tablas "overall" y "por paso" compatibles con los pivotes actuales.

In [1]:
# ===========================
# MÉTRICAS: registry modular
# ===========================
import numpy as np
import pandas as pd
from typing import Callable, Dict, Iterable

# --- utilidades de seguridad numérica ---
_EPS = 1e-12

def mae(y, yhat) -> float:
    y = np.asarray(y, float)
    yhat = np.asarray(yhat, float)
    return float(np.mean(np.abs(yhat - y)))

def smape(y, yhat) -> float:
    """sMAPE estable (0 si y = yhat = 0). Escala 0..200."""
    y = np.asarray(y, float)
    yhat = np.asarray(yhat, float)
    den = np.abs(y) + np.abs(yhat)
    num = 200.0 * np.abs(yhat - y)
    out = np.where(den < _EPS, 0.0, num / (den + _EPS))
    return float(np.mean(out))

def mase(y, yhat, insample = None, m: int = 1) -> float:
    """
    MASE = MAE_forecast / MAE_naive_in-sample estacional (lag m).
    - y: test
    - insample: serie de entrenamiento (requerida)
    """
    if insample is None or len(insample) <= m:
        return np.nan
    y = np.asarray(y, float); yhat = np.asarray(yhat, float)
    insample = np.asarray(insample, float)
    denom = np.mean(np.abs(insample[m:] - insample[:-m]))
    if denom < _EPS:
        return np.nan
    return float(np.mean(np.abs(yhat - y)) / denom)

def rmsse(y, yhat, insample = None, m: int = 1) -> float:
    """
    RMSSE = RMSE_forecast / RMSE_naive_in-sample estacional (lag m).
    """
    if insample is None or len(insample) <= m:
        return np.nan
    y = np.asarray(y, float)
    yhat = np.asarray(yhat, float)
    insample = np.asarray(insample, float)
    denom = np.mean((insample[m:] - insample[:-m])**2)
    if denom < _EPS:
        return np.nan
    rmse = np.sqrt(np.mean((yhat - y)**2))
    return float(rmse / np.sqrt(denom))

# --- registry: añade o quita métricas sin tocar el pipeline ---
METRICS_REGISTRY: Dict[str, Callable] = {
    "MAE":   mae,
    "sMAPE": smape,
    "MASE":  mase,  # requiere insample y m
    "RMSSE": rmsse, # requiere insample y m
}

# -----------------------------------------------
# Constructores de tablas largo → compatibles
# -----------------------------------------------
def build_metrics_overall_long(
        y_true:        np.ndarray,
        preds:         dict[str, np.ndarray],
        metrics:       Iterable[str] = ("MAE", "sMAPE"),
        *,
        insample:      np.ndarray | None = None,
        season_length: int = 1,
) -> pd.DataFrame:
    """
    Devuelve filas (model, metric, value).
    Si la métrica necesita insample/m, se pasan aquí.
    """
    rows = []
    for model, yhat in preds.items():
        for mname in metrics:
            fn = METRICS_REGISTRY[mname]
            if mname in ("MASE", "RMSSE"):
                val = fn(y_true, yhat, insample = insample, m = season_length)
            else:
                val = fn(y_true, yhat)
            rows.append({"model": model, "metric": mname, "value": float(val)})
    return pd.DataFrame(rows)

def build_metrics_by_step_long(
    y_true:  np.ndarray,
    preds:   dict[str, np.ndarray],
    metrics: Iterable[str] = ("MAE", "sMAPE"),
    aggs:    Iterable[str] = ("mean", "sum")
) -> pd.DataFrame:
    """
    Devuelve filas (model, h_step, metric, agg, value).
    Nota: por serie única, 'mean' y 'sum' por paso son idénticos;
    al agregar muchas series, diferenciarán.
    """
    rows = []
    H = len(y_true)
    for model, yhat in preds.items():
        yhat = np.asarray(yhat, float)
        # pre-cálculos por paso para métricas por punto
        abs_err = np.abs(y_true - yhat)
        smape_step = np.where(
            (np.abs(y_true) + np.abs(yhat)) < _EPS, 0.0,
            200.0 * np.abs(y_true - yhat) / (np.abs(y_true) + np.abs(yhat) + _EPS)
        )
        per_step = {
            "MAE": abs_err,
            "sMAPE": smape_step,
        }
        for t in range(1, H + 1):
            for mname in metrics:
                if mname not in per_step:
                    # métricas no aditivas por punto: se pueden omitir a nivel step
                    continue
                v = float(per_step[mname][t-1])
                if "mean" in aggs:
                    rows.append({"model": model, "h_step": t, "metric": mname, "agg": "mean", "value": v})
                if "sum" in aggs:
                    rows.append({"model": model, "h_step": t, "metric": mname, "agg": "sum",  "value": v})
    return pd.DataFrame(rows)

In [3]:
# MINI CHECKLIST Y SMOKE TEST DE VALIDACIÓN
# Datos de juguete
y_true = np.array([10, 0, 5, 8], dtype=float)
preds = {
    "M1": np.array([8, 0, 7, 8], dtype=float),
    "M2": np.array([10, 2, 5, 6], dtype=float),
}
insample = np.array([3, 5, 2, 6, 4, 7, 3, 9, 1, 0, 2, 4, 6, 8], dtype=float)  # p.ej. train

# Tablas “long”
overall = build_metrics_overall_long(
    y_true, preds, metrics=("MAE","sMAPE","MASE","RMSSE"),
    insample=insample, season_length=12
)
bystep = build_metrics_by_step_long(
    y_true, preds, metrics=("MAE","sMAPE"), aggs=("mean","sum")
)

display(overall)  # columnas: model, metric, value
display(bystep)   # columnas: model, h_step, metric, agg, value


,model,metric,value
0,M1,MAE,1.000000
1,M1,sMAPE,13.888889
2,M1,MASE,0.333333
3,M1,RMSSE,0.471405
4,M2,MAE,1.000000
5,M2,sMAPE,57.142857
6,M2,MASE,0.333333
7,M2,RMSSE,0.471405


,model,h_step,metric,agg,value
0,M1,1,MAE,mean,2.000000
1,M1,1,MAE,sum,2.000000
2,M1,1,sMAPE,mean,22.222222
3,M1,1,sMAPE,sum,22.222222
4,M1,2,MAE,mean,0.000000
5,M1,2,MAE,sum,0.000000
6,M1,2,sMAPE,mean,0.000000
7,M1,2,sMAPE,sum,0.000000
8,M1,3,MAE,mean,2.000000
9,M1,3,MAE,sum,2.000000


**Cómo se usa** (conceptual):
- **Se tiene** `y_true` (test), `preds = {'SF_NAIVE': yhat1, 'PID_Croston': yhat2, ...}` y `insample = train['y']`.
- Llamas a `build_metrics_overall_long(...)` y `build_metrics_by_step_long(...)`.
- Ya salen en formato **largo** listo para tus funciones `build_overall_table` y `build_step_tables`.
- Para añadir una métrica nueva, implementas la función y la registras en `METRICS_REGISTRY`.

---

## 2. SPLITTER ÚNICO (Expanding / Rolling / Mix)
- Produce folds coherentes con horizonte `H`.
- Parametriza: tamaño inicial, ventana (para rolling), paso, y el "punto de cambio" para el mix.
- **No entrena ni predice**; solo entrega índices o cortes temporales.

In [4]:
# ===========================
# TIME SPLITTER: único
# ===========================
from dataclasses import dataclass
from typing import Iterator, Literal, Sequence

SplitStrategy = Literal["expanding", "rolling", "mix"]

@dataclass
class Fold:
    train_idx: slice  # o (start, end)
    test_idx:  slice  # exactamente H puntos tras train_idx

def time_splitter(
        n: int,
        *,
        H: int,
        strategy: SplitStrategy = "expanding",
        initial_train: int | None = None,
        window: int | None = None,
        step: int = 1,
        mix_switch_train_len: int | None = None,
) -> Iterator[Fold]:
    """
    Genera folds (train_idx, test_idx) sobre una serie de longitud n.
    - expanding: train crece desde initial_train.
    - rolling:   train es ventana fija 'window'.
    - mix:       expanding hasta train_len == mix_switch_train_len, luego rolling.
    Requiere que test tenga H puntos completos.
    """
    if initial_train is None:
        # regla conservadora: al menos 3*H de arranque
        initial_train = max(H * 3, 12)
    
    last_test_end = n
    max_train_end = last_test_end - H # último índice que ún deja H puntos de test

    if strategy == "expanding":
        t = initial_train
        while t <= max_train_end:
            yield Fold(slice(0, t), slice(t, t + H))
            t += step
    
    elif strategy == "rolling":
        if window is None:
            # por defecto, ventana de 3*H
            window = max(H * 3, 12)
        t = window
        while t <= max_train_end:
            yield Fold(slice(t - window, t), slice(t, t + H))
            t += step
    
    elif strategy == "mix":
        if mix_switch_train_len is None:
            # por defecto: cambiar cuando train alcance 6*H
            mix_switch_train_len = max(H * 6, initial_train)
        # Fase 1: expanding
        t = initial_train
        while t < min(mix_switch_train_len, max_train_end + 1):
            yield Fold(slice(0, t), slice(t, t + H))
            t += step
        # Fase 2: rolling con ventana = mix_switch_train_len
        window = mix_switch_train_len
        while t <= max_train_end:
            yield Fold(slice(t - window, t), slice(t, t + H))
            t += step
    
    else:
        raise ValueError(f"Estrategia desconocida: {strategy}")

In [5]:
# SMOKE TESTS DEL SPLITTER PARA VALIDACIÓN
# Serie ficticia de longitud n
n, H = 60, 12

# Expanding
folds_exp = list(time_splitter(n=n, H=H, strategy="expanding", initial_train=36, step=1))
print("Expanding:", len(folds_exp), "folds",
      " | primer train:", folds_exp[0].train_idx, "test:", folds_exp[0].test_idx)

# Rolling (ventana fija)
folds_roll = list(time_splitter(n=n, H=H, strategy="rolling", window=36, step=1))
print("Rolling:", len(folds_roll), "folds",
      " | primer train:", folds_roll[0].train_idx, "test:", folds_roll[0].test_idx)

# Mix (expanding→rolling)
folds_mix = list(time_splitter(n=n, H=H, strategy="mix", initial_train=36, mix_switch_train_len=48, step=1))
print("Mix:", len(folds_mix), "folds",
      " | primer train:", folds_mix[0].train_idx, "test:", folds_mix[0].test_idx)


Expanding: 13 folds  | primer train: slice(0, 36, None) test: slice(36, 48, None)
Rolling: 13 folds  | primer train: slice(0, 36, None) test: slice(36, 48, None)
Mix: 13 folds  | primer train: slice(0, 36, None) test: slice(36, 48, None)


In [6]:
for name, folds in [
    ("Expanding", folds_exp),
    ("Rolling", folds_roll),
    ("Mix", folds_mix)
]:
    print(f"\n{name}:")
    for i, f in enumerate(folds[:3], start=1):  # solo 3 primeros folds
        print(f"  Fold {i} train: {f.train_idx}, test: {f.test_idx}")

# Expanding:
#   Fold 1 train: slice(0, 36, None), test: slice(36, 48, None)
#   Fold 2 train: slice(0, 37, None), test: slice(37, 49, None)
#   Fold 3 train: slice(0, 38, None), test: slice(38, 50, None)

# Rolling:
#   Fold 1 train: slice(0, 36, None), test: slice(36, 48, None)
#   Fold 2 train: slice(1, 37, None), test: slice(37, 49, None)
#   Fold 3 train: slice(2, 38, None), test: slice(38, 50, None)

# Mix:
#   Fold 1 train: slice(0, 36, None), test: slice(36, 48, None)
#   Fold 2 train: slice(0, 37, None), test: slice(37, 49, None)
#   Fold 3 train: slice(0, 38, None), test: slice(38, 50, None)

# Esto confira que el spliiter funciona tal y como se diseñó:
# Expanding --> siempre empieza en 0 y el tamaño de train crece
#               (stop sube: 36 -> 37 -> 38...).
# Rolling ----> ventana fija (aquí 36), pero se va desplazando 
#               (start y stop) suben juntos: 0-36 -> 1-37 -> 2-38...).
# Mix --------> empieza como expanding (primeros folds como en expanding) y luego, 
#               cuando aclanza el mix_switch_train_len, pasará a modo rolling.




Expanding:
  Fold 1 train: slice(0, 36, None), test: slice(36, 48, None)
  Fold 2 train: slice(0, 37, None), test: slice(37, 49, None)
  Fold 3 train: slice(0, 38, None), test: slice(38, 50, None)

Rolling:
  Fold 1 train: slice(0, 36, None), test: slice(36, 48, None)
  Fold 2 train: slice(1, 37, None), test: slice(37, 49, None)
  Fold 3 train: slice(2, 38, None), test: slice(38, 50, None)

Mix:
  Fold 1 train: slice(0, 36, None), test: slice(36, 48, None)
  Fold 2 train: slice(0, 37, None), test: slice(37, 49, None)
  Fold 3 train: slice(0, 38, None), test: slice(38, 50, None)


**Cómo se usa** (conceptual):
- Para una serie de longitud `n = len(y)` y `H = 12`:
    - `for fold in time_splitter(n, H = 12, strategy = 'expanding', initial_train = 36, step = 1): ...`
    - Cada `fold` te da `train_idx` y `test_idx` (exactos `H` puntos) que luego aplicas sobre tus arrays/DF.

**¿Qué ganamos ahora?**
- Se puede **enchufar cualquier modelo** y calcular **cualquier métrica** sin tocar el pipeline.
- El splitter único evita reescribir lógica de ventanas en cada bloque.
- Ya está listo para:
    - Cambiar definiciones de métricas cuando se desee.
    - Añadir nuevas (propias o de librerías) con un wrapper.
    - Usar las **mismas funciones de tabulado** que venimos usando.

---

## 3. CONECTAR EL SPLITTER A VARIOS MODELOS Y CONSOLIDAR MÉTRICAS (media sobre folds)

Este bloque:
- Usa el splitter para generar folds (elige expanding/rolling/mix).
- Corre 4 modelos por fold:
    - SF_NAIVE, SF_SEASONALNAIVE y SF_CROSTON con statsforecast.
    - PID_Croston con pyInterDemand (si no está disponible, te da un error claro).
- Calcula métricas por fold con el registry de métricas (MAE, sMAPE; se puede añadir MASE/RMSSE pasando insample = train_y).
- Agrega por media sobre folds (overall y por paso).
- Deja listo para pasar a tus funciones de tablas anchas.

Supone que ya se tiene carga las utilidades del metrics registry y el splitter preparado en pasos anteriores (build_metrics_overall_long, build_metrics_by_step_long, time_splitter).

In [27]:
# ===========================================
#  FOLD RUNNER: SF_NAIVE, SF_SEASONALNAIVE,
#               SF_CROSTON, PID_Croston
#  - agrega métricas por media sobre folds
# ===========================================
import numpy as np
import pandas as pd
from typing import Literal
from typing import Any

from statsforecast import StatsForecast
from statsforecast.models import Naive as SF_Naive
from statsforecast.models import SeasonalNaive as SF_SNaive
from statsforecast.models import CrostonClassic as SF_Croston

# <- necesitas tener en el entorno:
# - time_splitter (estrategias: expanding/rolling/mix)
# - build_metrics_overall_long, build_metrics_by_step_long
# - METRICS_REGISTRY con al menos {"MAE", "sMAPE"}

def _fit_predict_sf(df_train: pd.DataFrame,
                    H: int,
                    freq: str,
                    model_obj,
                    col_name: str) -> pd.Series:
    """
    Ajusta un modelo de statsforecast y devuelve un pd.Series con 
    índice = fechas del test (los H pasos siguientes al último ds de train).
    """
    sf = StatsForecast(models = [model_obj], freq = freq, n_jobs = 1)
    fcst = sf.forecast(df = df_train, h = H)
    # la columna tiene el nomobre del modelo de SF (e.g. 'Naive'), la renombramos
    mcol = [c for c in fcst.columns if c not in ('unique_id', 'ds')]
    assert len(mcol) == 1, "Se esperaba una sola columna de predicción SF"
    fcst = fcst.rename(columns = {mcol[0]: col_name})
    return fcst.set_index('ds')[col_name]

# def _fit_predict_pid_croston(y_train: np.ndarray, H: int) -> np.ndarray:
#     """
#     Envuelve Croston de pyInterDemand.
#     Adapta si la API difiere.
#     Debe devolver un array de longitud H.
#     """
#     try:
#         # Ajusta estos imports si la instalación usa otros paths/nombres
#         from pyInterDemand.algorithm.intermittent import croston_method as PID_Croston
#     except Exception as e:
#         raise ImportError(
#             "No se puede importar Croston de pyInterDemand."
#             "Verifica instalación y ruta de import."
#         ) from e
    
#     # Suponiendo interfaz tipo clase con fit/predict; si es función, adapta:
#     mdl = PID_Croston()
#     # Si la clase necesita parámetos (alpha), pásalos al constructor.
#     yhat = mdl.fit_predict(y_train, H) if hasattr(mdl, "fit_predict") else None
#     if yhat is None:
#         # fallback genérico fit + predict
#         if hasattr(mdl, "fit") and hasattr (mdl, "predict"):
#             mdl.fit(y_train)
#             yhat = mdl.predict(H)
#         else:
#             # o si es una función, por ejemplo: yhat = Croston(y_train, H)
#             yhat = PID_Croston(y_train, H)
    
#     yhat = np.asarray(yhat, float)
#     if yhat.shape[0] != H:
#         raise ValueError(f"PID_Croston devolvió {len(yhat)} pasos, esperado H = {H}")
#     return yhat

# === REEMPLAZO de la función _fit_predict_pid_croston por ésta ===
def _fit_predict_pid_croston_df(
    train_df: pd.DataFrame,
    H: int,
    *,
    alpha: float = 0.1,
    freq: int = 1,  # usar entero para evitar lógicas con fechas
) -> np.ndarray:
    """
    Llama a pyInterDemand.algorithm.intermittent.croston_method asegurando:
      - ts con índice 0..n-1 (RangeIndex) para evitar KeyError por indexación por etiqueta.
      - normaliza la salida a un np.ndarray de longitud H.

    Parámetros:
      train_df: DataFrame con columnas ['ds','y'] (y opcionalmente 'unique_id')
      H: horizonte
      alpha: suavizado Croston
      freq: entero (1) para paso unitario
    """
    from pyInterDemand.algorithm.intermittent import croston_method as PID_Croston

    # Orden temporal y construcción de serie con índice posicional
    df_ = train_df.sort_values("ds").copy()
    ts = pd.Series(np.asarray(df_["y"], dtype=float))
    ts.index = pd.RangeIndex(start=0, stop=len(ts), step=1)  # índice 0..n-1

    # Llamada principal (firma estable en tu instalación)
    yhat = PID_Croston(ts=ts, alpha=alpha, n_steps=H, freq=freq)

    # ---- Normalización de la salida a un array 1D de tamaño H ----
    def _to_array(obj: Any) -> np.ndarray:
        # Si es lista/tupla y contiene Series/arrays, nos quedamos con el último (forecast)
        if isinstance(obj, (list, tuple)):
            if all(isinstance(el, pd.Series) for el in obj) and len(obj) > 0:
                return np.asarray(obj[-1].values, dtype=float).ravel()
            # si no, intenta convertir todo directamente
            return np.asarray(obj, dtype=float).ravel()
        if isinstance(obj, pd.Series):
            return np.asarray(obj.values, dtype=float).ravel()
        if isinstance(obj, np.ndarray):
            return obj.astype(float).ravel()
        if isinstance(obj, dict):
            for k in ("forecast", "yhat", "mean", "fcst", "fcast"):
                if k in obj:
                    return np.asarray(obj[k], dtype=float).ravel()
            raise ValueError("Salida dict de Croston sin claves forecast reconocidas.")
        # último recurso: intenta convertir a array
        return np.asarray(obj, dtype=float).ravel()

    arr = _to_array(yhat)

    # Si viniera fitted+forecast concatenado, recorta a los últimos H
    if arr.shape[0] > H:
        arr = arr[-H:]

    if arr.shape[0] != H:
        raise ValueError(f"Croston (PID) devolvió {arr.shape[0]} pasos, esperado H={H}.")

    return arr

# === DEBUG opcional para inspeccionar la salida real de Croston PID ===
def debug_pid_croston(
    train_df: pd.DataFrame, H: int, alpha: float = 0.1, freq: int = 1
):
    """
    Imprime la estructura exacta que devuelve croston_method, con índice posicional.
    Útil si cambias de versión y necesitas ajustar el wrapper.
    """
    from pyInterDemand.algorithm.intermittent import croston_method as PID_Croston

    df_ = train_df.sort_values("ds").copy()
    ts = pd.Series(np.asarray(df_["y"], dtype=float))
    ts.index = pd.RangeIndex(start=0, stop=len(ts), step=1)

    print(f"[debug] len(ts)={len(ts)}, H={H}, alpha={alpha}, freq={freq}")
    yhat = PID_Croston(ts=ts, alpha=alpha, n_steps=H, freq=freq)

    print("[debug] type(yhat):", type(yhat))
    if isinstance(yhat, pd.Series):
        print("[debug] Series len:", len(yhat))
        print("[debug] head:", yhat.head().to_string())
        print("[debug] tail:", yhat.tail().to_string())
    elif isinstance(yhat, (list, tuple)):
        print("[debug] container len:", len(yhat))
        for i, el in enumerate(yhat):
            try:
                L = len(el)
            except Exception:
                L = "NA"
            print(f"  - item[{i}] type={type(el)} len={L}")
            try:
                arr = np.asarray(el, dtype=float).ravel()
                print("    head:", arr[:5], "| tail:", arr[-5:])
            except Exception:
                pass
    elif isinstance(yhat, dict):
        print("[debug] dict keys:", list(yhat.keys()))
        for k, v in yhat.items():
            try:
                L = len(v)
            except Exception:
                L = "NA"
            print(f"  - {k}: type={type(v)} len={L}")
    else:
        print("[debug] valor no reconocido; repr:", repr(yhat))

    return yhat

In [35]:
def run_xmodels_over_folds(
        df: pd.DataFrame,
        *,
        freq:                 str = "MS",
        H:                    int = 12,
        strategy:             Literal["expanding", "rolling", "mix"] = "expanding",
        initial_train:        int | None = None,
        window:               int | None = None,
        step:                 int = 1,
        mix_switch_train_len: int | None = None,
        unique_id:            str | None = None,
        season_length:        int = 12,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    df: DataFrame con columnas ['unique_id', 'ds', 'y'] de UNA serie (un único unique_id).
    Devuelve:
        - metrics_overall_mean: media sobre folds de (model, metric, value)
        - metrics_by_step_mean: media sobre folds de (model, h_step, metric, agg, value)
    """
    # ---- Chequeos y Preparación ----
    cols_req = {'unique_id', 'ds', 'y'}
    if not cols_req.issubset(df.columns):
        raise ValueError(f"df debe contener columnas {cols_req}")
    df = df.sort_values('ds').reset_index(drop = True).copy()

    if unique_id is None:
        uids = df['unique_id'].unique()
        assert len(uids) == 1, "Proporciona un df de una sola serie o indica unique_id."
        uid = uids[0]
    else:
        uid = unique_id
        df = df[df['unique_id'] == uid].copy()
        if df.empty:
            raise ValueError(f"No hay datos para unique_id = {uid} en el df suministrado.")
    
    y = df['y'].astype(float).values
    n = len(y)

    # ---- Generar folds con Splitter ----
    folds = list(time_splitter(n = n,
                               H = H,
                               strategy = strategy,
                               initial_train = initial_train,
                               window = window,
                               step = step,
                               mix_switch_train_len = mix_switch_train_len
                               )
                )
    if len(folds) == 0:
        raise ValueError("El splitter no generó folds. Revisa 'initial_train', 'window' y 'H'.")
    
    # ---- Acumular métricas por Fold ----
    metrics_overall_all = []
    metrics_by_step_all = []

    for k, f in enumerate(folds, start = 1):
        train = df.iloc[f.train_idx].copy()
        test =  df.iloc[f.test_idx].copy()
        y_train = train['y'].astype(float).values
        y_test  = test['y'].astype(float).values

        # ========== MODELOS ==========

        # 1) StatsForecast Naïve (SF_Naive)
        sf_naive = _fit_predict_sf(train, 
                                   H = H, 
                                   freq = freq, 
                                   model_obj = SF_Naive(), 
                                   col_name = 'SF_Naive')
        
        # 2) StatsForecast - Seasonal Naïve (SF_SNaive)
        sf_season = _fit_predict_sf(train,
                                    H = H,
                                    freq = freq,
                                    model_obj = SF_SNaive(season_length = season_length),
                                    col_name = 'SF_SeasonalNaive')
        
        # 3) StatsForecast - Croston (SF_Croston)
        sf_cro = _fit_predict_sf(train, 
                                 H = H,
                                 freq = freq,
                                 model_obj = SF_Croston(),
                                 col_name = 'SF_Croston')
        
        # 4) pyInterDemand - Croston (PID_Croston) usando wrapper robusto
        # pid_cro = _fit_predict_pid_croston_df(train, H=H, alpha=0.1, freq=1)
        # pid_cro = pd.Series(pid_cro, index=test['ds'], name='PID_Croston')


        # Alinear preds en un dict homogéneo np.ndarray
        preds = {
            'SF_Naive':         sf_naive.reindex(test['ds']).values,
            'SF_SeasonalNaive': sf_season.reindex(test['ds']).values,
            'SF_Croston':       sf_cro.reindex(test['ds']).values,
            # 'PID_Croston':      pid_cro.reindex(test['ds']).values
        }

        # ========== MÉTRICAS DEL FOLD ==========
        # Overall (por horizonte completo); pasa insample para MASE/RMSSE si se activan
        mo = build_metrics_overall_long(
            y_true = y_test,
            preds = preds,
            metrics = ('MAE', 'sMAPE'),
            insample = y_train,
            season_length = season_length
        )
        mo['fold'] = k

        # por paso (mean/sum) - útil para ver degradación vs h
        ms = build_metrics_by_step_long(
            y_true = y_test,
            preds = preds,
            metrics = ('MAE', 'sMAPE'),
            aggs = ('mean', 'sum')
        )
        ms['fold'] = k

        metrics_overall_all.append(mo)
        metrics_by_step_all.append(ms)
    
    metrics_overall_all = pd.concat(metrics_overall_all, ignore_index = True)
    metrics_by_step_all = pd.concat(metrics_by_step_all, ignore_index = True)

    # ---- Agregación (Media sobre Folds) ----
    metrics_overall_mean = (
        metrics_overall_all
        .groupby(['model', 'metric'], as_index = False)['value']
        .mean()
        .rename(columns = {'value': 'value_mean'})
    )

    metrics_by_step_mean = (
        metrics_by_step_all
        .groupby(['model', 'h_step', 'metric', 'agg'], as_index = False)['value']
        .mean()
        .rename(columns = {'value': 'value_mean'})
    )

    return metrics_overall_mean, metrics_by_step_mean

In [36]:
# CHECKLIST DE PRUEBAS RÁPIDAS DE VERIFICACIÓN Y VALIDACIÓN
# ====== SMOKE TEST RÁPIDO ======
import numpy as np
import pandas as pd

# serie mensual 72 puntos con intermitencia
rng = pd.date_range('2018-01-01', periods=72, freq='MS')
y = (np.sin(np.arange(72)/6)*50 + 200).clip(0)
y[(np.arange(72) % 7) == 0] = 0

df_demo = pd.DataFrame({'unique_id': 'SERIE_DEMO', 'ds': rng, 'y': y})

H = 12
m_overall, m_by_step = run_xmodels_over_folds(
    df_demo,
    freq='MS',
    H=H,
    strategy='expanding',
    initial_train=36,   # o 12 si quieres apretar, pero 36 da más holgura
    step=1,
    season_length=12
)
print("OK: función ejecutada.")
display(m_overall.head())
display(m_by_step.head())


OK: función ejecutada.


,model,metric,value_mean
0,SF_Croston,MAE,67.096480
1,SF_Croston,sMAPE,49.793983
2,SF_Naive,MAE,81.937886
3,SF_Naive,sMAPE,65.840021
4,SF_SeasonalNaive,MAE,100.685852


,model,h_step,metric,agg,value_mean
0,SF_Croston,1,MAE,mean,63.577892
1,SF_Croston,1,MAE,sum,63.577892
2,SF_Croston,1,sMAPE,mean,44.816028
3,SF_Croston,1,sMAPE,sum,44.816028
4,SF_Croston,2,MAE,mean,63.597987


In [38]:
# 2) VALIDACIÓN DE ESQUEMA Y CONTENIDO
# columnas esperadas
assert set(m_overall.columns) == {'model', 'metric', 'value_mean'}, m_overall.columns
assert set(m_by_step.columns) == {'model', 'h_step', 'metric', 'agg', 'value_mean'}, m_by_step.columns

# modelos esperados presentes
expected_models = {'SF_Naive', 'SF_SeasonalNaive', 'SF_Croston'}
assert expected_models.issubset(set(m_overall['model'])), set(m_overall['model'])

# métricas esperadas
assert{'MAE', 'sMAPE'}.issubset(set(m_overall['metric'])), set(m_overall['metric'])

# pasos de horizonte completos (1...H)
steps_present = set(m_by_step['h_step'].unique())
assert steps_present == set(range(1, H + 1)), steps_present

# No NaNs "raros"
assert m_overall['value_mean'].notna().all()
assert m_by_step['value_mean'].notna().all()

print('OK: Validaciones de esquema y contenido superadas.')

OK: Validaciones de esquema y contenido superadas.


In [39]:
# 3) SANITY CHECK BÁSICO DE MAGNITUDES
# sMAPE debe estar en [0, 200]
s = m_overall.query("metric=='sMAPE'")['value_mean']
assert (s.between(0, 200)).all(), s.describe()

# MAE no negativa
a = m_overall.query("metric=='MAE'")['value_mean']
assert (a >= 0).all(), a.describe()

print('OK: Sanity checks de métricas superados.')

OK: Sanity checks de métricas superados.


In [43]:
# 4) VISTA RÁPIDA DE RESULTADOS
print('\n=== Overall (media sobre folds) ===')
display(m_overall.pivot(index = 'model', columns = 'metric', values = 'value_mean'))

print('\n=== MAE por paso (media sobre folds) ===')
display(
    m_by_step[(m_by_step["metric"] == "MAE") & (m_by_step["agg"] == "mean")]
    .pivot(index="model", columns="h_step", values="value_mean")
)

# ¿Qué se debería ver?
# - La ejecución termina sin errores.
# - m_overall contiene 4 filas (una por modelo) x 2 métricas (MAE, sMAPE) agregadas por media de folds.
# - m_by_step tiene 4 modelos x H pasos x 2 métricas x 2 aggs (mean/sum).
# - Las validaciones imprimen 'OK...'.



=== Overall (media sobre folds) ===


metric,MAE,sMAPE
model,,
SF_Croston,67.096480,49.793983
SF_Naive,81.937886,65.840021
SF_SeasonalNaive,100.685852,77.139997



=== MAE por paso (media sobre folds) ===


h_step,1,2,3,4,5,6,7,8,9,10,11,12
model,,,,,,,,,,,,
SF_Croston,63.577892,63.597987,63.997401,70.320382,69.951339,69.035907,71.326505,67.932645,66.077441,64.502912,68.749063,66.088287
SF_Naive,66.427015,70.687223,74.716139,84.601003,87.759603,90.424199,31.007838,90.177652,92.196476,93.642988,100.808675,100.805822
SF_SeasonalNaive,91.517921,97.211784,99.086610,106.725301,107.607895,101.948450,101.741650,98.140807,100.910747,99.149696,103.383542,100.805822


**Cómo se usa** (conceptual):
1. Selecciona un **dataframe de una sola serie** (o filtra por `unique_id`).
2. Llama a `run_xmodels_over_folds(df_series, freq = 'MS', H = 12, strategy = 'expanding', initial_train = 36, step = 1, season_length = 12)`.
3. Obtendrás:
    - `metrics_overall_mean` → (model, metric, value_mean) **ya promediado por folds**.
    - `metrics_by_step_mean` → (model, h_step, metric, agg, value_mean) **promedio por paso y fold**.
  Luego se puede:
    - Pasar metrics_overall_mean.rename(columns = {'value_mean': 'value'}) a build_overall_table(...).
    - Pasar metrics_by_step_mean.rename(columns = {'value_mean': 'value'}) a build_step_table(...).

---

## BLOQUES
### B1 - Chequeos, selección automática de DF e ID
### B2 - Ejecución de runner y construcción de tablas
### B3 - Gráfica de inspección del último fold

Así se garantiza que:
- Primero se cargan todas las funciones y heplers que usa todo lo demás.
- Luego se selecciona automáticamente eld ataset y parámetros (H, SERIE_ID).
- Después se corre el backtest y se produce las tablas.
- Finalmente se genera el gráfico para visualizar un fold concreto.

In [44]:
# ===========================================
#  BLOQUE 1:
#  Chequeos, selección de df_* y unique_id
# ===========================================

import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ====== 1) pyInterDemand: si no está, desactiva PID_Croston con NaN ======
try:
    from pyInterDemand.algorithm.intermittent import croston_method as _PID_CHECK # noqa: F401
    PID_AVAILABLE = True
except Exception:
    PID_AVAILABLE = False

if not PID_AVAILABLE:
    print('⚠️ pyInterDemand NO está disponible: PID_Croston se ignorará (NaN).')
    # Sobreescribe helper para que no rompa
    def _fit_predict_pid_croston(y_train: np.ndarray, H: int) -> np.ndarray: # noqa: F811
        return np.full(H, np.nan, dtype = float)


# ====== 2) Elegir automáticamente un df_* y extraer H del nombre ======
# Preferencia por h12 -> h6 -> h3
_pat = re.compile(r"^df_.+_h(3|6|12)$")

candidatos = []
for name, obj in globals().items():
    if isinstance(obj, pd.DataFrame) and name.startswith("df_") and _pat.match(name):
        candidatos.append(name)

preferencia = sorted(
    candidatos,
    key = lambda n: ({'12':0, '6':1, '3':2}[re.search(r"h(3|6|12)$", n).group(1)], n)
)

if not preferencia:
    raise RuntimeError("No se encontró ningún DataFrame 'df_*_h{3|6|12}' en el entorno.")

DF_NAME = preferencia[0]
df = globals()[DF_NAME].copy()
print(f"✅ Usando DataFrame: {DF_NAME} → shape = {df.shape}")

# Extraer H del nombre
_Hm = re.search(r"h(3|6|12)$", DF_NAME)
H = int(_Hm.group(1)) if _Hm else 12
print(f"Horizonte detectado H = {H}")


# ====== 3) Elegir una unique_id válida para backtesting (n >= initial_train + H) ======
# Con el splitter expanding por defecto, initial_train = max(3H, 12).
# Se requiere n >= 4H como mínimo.
need_min = max(4 * H, 12 + H)
counts = df.groupby('unique_id', as_index = False)['y'].count().rename(columns = {'y':'n'})
candidatos_uid = counts[counts['n'] >= need_min].sort_values('n', ascending = False)

if candidatos_uid.empty:
    # Si no hay ninguna con >= 4H, intenta con >= (3H - H) = 4H equivalente; si aún no hay, escoge la más larga
    candidatos_uid = counts.sort_values('n', ascending = False).head(1)

SERIE_ID = candidatos_uid.iloc[0]['unique_id']
print(f'✅ Elegida unique_id = {SERIE_ID} con n = {int(candidatos_uid.iloc[0]['n'])}')

# Filtrar a una sola serie (el runner lo admite así)
df_one = df[df['unique_id'] == SERIE_ID].sort_values('ds').reset_index(drop = True)

RuntimeError: No se encontró ningún DataFrame 'df_*_h{3|6|12}' en el entorno.

In [ ]:
# ===========================================
#  BLOQUE 2:
#  Ejecutar runner y construir tablas
# ===========================================

# Parámetros del split/backtesting
FREQ = "MS"            # Mensual
SEASON_LENGTH = 12     # Estacionalidad mensual
STRATEGY = "expanding" # Se puede cambiar a 'rolling' o 'mix'
INITIAL_TRAIN = None   # Usa la regla por defecto (max(3H, 12))
STEP = 1


# ====== 1) Ejecutar folds y consolidar métricas (media sobre folds)
metrics_overall_mean, metrics_by_step_mean = run_xmodels_over_folds(
    df_one,
    freq = FREQ,
    H = H,
    strategy = STRATEGY,
    initial_train = INITIAL_TRAIN,
    step = STEP,
    season_length = SEASON_LENGTH
)


# ====== 2) Construir tablas con tus helpers (renombrando 'value_mean' → 'value')
metrics_overall_mean_ = metrics_overall_mean.rename(columns = {'value_mean':'value'})
metrics_by_step_mean_ = metrics_by_step_mean.rename(columns = {'value_mean':'value'})

tabla_general = build_overall_table(metrics_overall_mean_)
tablas_por_metric = build_step_tables(
    metrics_by_step = metrics_by_step_mean_,
    H = H,
    metrics = None,
    aggs = ('mean', 'sum')
)

print('=== TABLA GENERAL (media sobre folds) ===')
display(tabla_general)

for key, df_tab in tablas_por_metric.items():
    print(f'\n=== Tabla {key} (media sobre folds) ===')
    display(df_tab)

In [ ]:
# ===========================================
#  BLOQUE 3:
#  Gráfica de inspección del último fold
#  (train cola + test + 4 modelos)
# ===========================================

# Reconstrucción del último fold para graficar predicciones explícitamente
folds = list(time_splitter(n = len(df_one), H = H, strategy = STRATEGY,
                           initial_train = INITIAL_TRAIN, step = STEP))
last_fold = folds[-1]

train = df_one.iloc[last_fold.train_idx].copy()
test  = df_one.iloc[last_fold.test_idx].copy()

from statsforecast.models import Naive as SF_Naive
from statsforecast.models import SeasonalNaive as SF_SNaive
from statsforecast.models import CrostonClassic as SF_Croston

# Predicciones por modelo en el último fold
sf_naive  = _fit_predict_sf(train, H = H, freq = FREQ, model_obj = SF_Naive(), col_name = 'SF_Naive')
sf_snaive = _fit_predict_sf(train, H = H, freq = FREQ, model_obj = SF_SNaive(), col_name = 'SF_SNaive')
sf_cros   = _fit_predict_sf(train, H = H, freq = FREQ, model_obj = SF_Croston(), col_name = 'SF_Croston')
pid_cros  = pd.Series(_fit_predict_pid_croston(train['y'].values.astype(float), H = H), index = test['ds'], name = 'PID_Croston')

# Ensamblar para plot
comp = (test[['ds', 'y']]
        .merge(sf_naive.rename('SF_Naive'), left_on = 'ds', right_index = True, how = 'left')
        .merge(sf_snaive.rename('SF_SeasonalNaive'), left_on = 'ds', right_index = True, how = 'left')
        .merge(sf_cros.rename('SF_Croston'), left_on = 'ds', right_index = True, how = 'left')
        .merge(pid_cros, left_on = 'ds', right_index = True, how = 'left'))

# Gráfico con tu estilo
plt.figure(fgisize = (12, 6))
plt.plot(train['ds'].tail(36), train['y'].tail(36), label = 'Train (cola)', marker = 'o')
plt.plot(test['ds'].tail(36), test['y'].tail(36), label = 'Test (cola)', marker = 'o')

for m in ['SF_Naive', 'SF_SeasonalNaive', 'SF_Croston', 'PID_Croston']:
    if m in comp.columns:
        plt.plot(comp['ds'], comp[m], label = f'Forecast - {m}', marker = 'x')

plt.title(f'Backtest (último fold) - H = {H} - Serie {SERIE_ID}')
plt.xlabel('Fecha')
plt.ylabel('Valor')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

**Qué esperar**:
- **Tablas**: una **general** (métrica agregada media sobre folds por modelo) y tantas **tablas por paso** como combinaciones `métricaxagg` existan en `metrics_by_step_mean_` (por defecto, MAE/sMAPE x mean/sum).
- **Gráfica**: compara visualmente *train cola, test* y los **4 modelos** en el **último fold**.
- Si **pyInterDemand** no está disponible, verás una advertencia y el modelo **PID_Croston** aparecerá con `NaN` en métricas/plot (línea ausente).

---

Si funciona, en el siguiente micro-paso se integra rolling y mix (mismos artefactos) y se añade la persistencia (Parquet) de métricas y metadatos del experimento.

In [23]:
%pip install pycaret-ts-alpha

     ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
     ----- ---------------------------------- 2.4/15.8 MB 11.9 MB/s eta 0:00:02
     ----- ---------------------------------- 2.4/15.8 MB 11.9 MB/s eta 0:00:02
     ---------- ----------------------------- 4.2/15.8 MB 6.7 MB/s eta 0:00:02
     ------------- -------------------------- 5.5/15.8 MB 7.5 MB/s eta 0:00:02
     --------------- ------------------------ 6.0/15.8 MB 5.8 MB/s eta 0:00:02
     --------------------- ------------------ 8.4/15.8 MB 6.8 MB/s eta 0:00:02
     --------------------- ------------------ 8.4/15.8 MB 6.8 MB/s eta 0:00:02
     --------------------- ------------------ 8.4/15.8 MB 6.8 MB/s eta 0:00:02
     --------------------- ------------------ 8.4/15.8 MB 6.8 MB/s eta 0:00:02
     -------------------------- ------------- 10.5/15.8 MB 5.0 MB/s eta 0:00:02
     --------------------------------- ------ 13.1/15.8 MB 5.6 MB/s eta 0:00:01
     --------------------------------- ------ 13.1/15.8

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\AlejandroLaderasImpe\\AppData\\Local\\Temp\\pip-install-4fnp3a08\\numpy_053d708d49fc46b7baafabc8b43b5f18\\vendored-meson\\meson\\test cases\\linuxlike\\13 cmake dependency\\cmake_pref_env\\lib\\cmake\\cmMesonVersionedTestDep\\cmMesonVersionedTestDepConfigVersion.cmake'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
%pip install pycaret[full]

  Using cached pycaret-3.3.2-py3-none-any.whl.metadata (17 kB)
  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached numpy-1.26.4.tar.gz (15.8 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\AlejandroLaderasImpe\\AppData\\Local\\Temp\\pip-install-zyyfs1oq\\numpy_e4c87ff69b58428188de684d0dbcba17\\vendored-meson\\meson\\test cases\\linuxlike\\13 cmake dependency\\cmake_pref_env\\lib\\cmake\\cmMesonVersionedTestDep\\cmMesonVersionedTestDepConfigVersion.cmake'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# CONJUNTO DE DATOS
data = get_data('airlineer')


# Este código importa la función get_data del módulo pycaret.datasets y la utiliza para cargar el conjunto de datos airlineer en una variable llamada data. 
# La función get_data es una función de utilidad proporcionada por la biblioteca PyCaret que permite a los usuarios cargar fácilmente conjuntos de datos 
# preexistentes para utilizarlos en tareas de aprendizaje automático. El conjunto de datos "airlineer" es un conjunto de datos de ejemplo incluido en la 
# biblioteca PyCaret que contiene información sobre pasajeros de líneas aéreas y los detalles de sus vuelos.

ModuleNotFoundError: No module named 'pycaret'